In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math
import folium

In [9]:
df = pd.read_csv('data/source/labels-with-tags-chu.csv')
#select city == seattle
df = df[df['city'] == 'seattle']
df

,city,label_id,label_type,severity,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,global_attribute_id,tag_list,description,lat,lng
0,seattle,161174,CurbRamp,1.0,1,317.000000,-16.250000,270.391571,-0.427856,8b57878d-8187-4c0a-b7f2-9a661d264985,NaN,missing tactile warning,NaN,47.665028,-122.313225
1,seattle,161176,CurbRamp,1.0,1,47.446430,-15.982142,1.767984,-0.627083,8b57878d-8187-4c0a-b7f2-9a661d264985,68677898.0,NaN,NaN,47.664982,-122.312920
2,seattle,188676,CurbRamp,1.0,1,221.437500,-35.000000,164.554047,0.114868,9a30e15b-c0bb-4e11-b801-bbab5da9c789,68708456.0,NaN,NaN,47.597446,-122.309044
3,seattle,161154,Obstacle,2.0,1,273.696442,-17.142857,78.104256,-5.573174,8b57878d-8187-4c0a-b7f2-9a661d264985,68679755.0,NaN,NaN,47.663170,-122.312782
4,seattle,161154,Obstacle,2.0,1,273.696442,-17.142857,78.104256,-5.573174,8b57878d-8187-4c0a-b7f2-9a661d264985,68680859.0,NaN,NaN,47.663170,-122.312782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266085,seattle,234196,NoSidewalk,4.0,1,332.750000,-18.875000,168.048965,0.996870,9d3db893-2f80-4dd4-85c3-43280cbc7a66,68727209.0,street has no sidewalks,NaN,47.692036,-122.367393
266086,seattle,234213,CurbRamp,1.0,1,266.812500,-25.000000,25.394808,-1.345330,9d3db893-2f80-4dd4-85c3-43280cbc7a66,68642579.0,NaN,NaN,47.677555,-122.279671
266087,seattle,234214,CurbRamp,1.0,1,266.812500,-25.000000,25.394808,-1.345330,9d3db893-2f80-4dd4-85c3-43280cbc7a66,68642572.0,NaN,NaN,47.677578,-122.279709
266088,seattle,234312,Signal,NaN,2,246.171875,-7.513393,88.950325,-0.363716,e4ff325c-309b-4ea6-8555-8f973e662362,68726853.0,NaN,NaN,47.686874,-122.366241


In [10]:
tags = df[['label_id','label_type','severity','tag_list']]

In [11]:
tags = tags.replace(to_replace= r'\\', value= '', regex=True)
tags = tags.replace(to_replace= r'"', value= '', regex=True)
tags['tag_list'] = tags['tag_list'].str.strip('[]')
tags['tag_list'] = tags['tag_list'].str.rstrip(' ')
tags['tag_list'] = tags['tag_list'].str.split(',')
tags = tags.explode('tag_list')
tags['tag_list'] = tags['tag_list'].str.lstrip(' ')
tags = tags.replace('', 'n/a')

In [13]:
#drop nan in tag_list
tags = tags.dropna(subset=['tag_list'])

In [14]:
tags

,label_id,label_type,severity,tag_list
0,161174,CurbRamp,1.0,missing tactile warning
7,173574,SurfaceProblem,2.0,grass
8,173574,SurfaceProblem,2.0,grass
9,173594,SurfaceProblem,1.0,cracks
10,173594,SurfaceProblem,1.0,cracks
...,...,...,...,...
266082,234180,SurfaceProblem,3.0,bumpy
266082,234180,SurfaceProblem,3.0,sand/gravel
266083,234180,SurfaceProblem,3.0,bumpy
266083,234180,SurfaceProblem,3.0,sand/gravel


In [30]:
#select only obstacle as label_type
o = tags[tags['label_type'] == 'Obstacle']
#add count column
o['count'] = 1

In [32]:
#group by tag_list sum
o = o.groupby('tag_list').sum()

In [34]:
#sort by count and also add a column for percentage
o = o.sort_values(by=['count'], ascending=False)
o['percentage'] = o['count']/o['count'].sum()*100
o

,label_id,label_type,severity,count,percentage
tag_list,,,,,
pole,4410,4410,4009,4410,21.044092
trash/recycling can,3748,3748,3462,3748,17.885093
vegetation,3265,3265,3060,3265,15.580263
parked car,3198,3198,2946,3198,15.260546
sign,1639,1639,1490,1639,7.821149
tree,1211,1211,880,1211,5.778775
fire hydrant,921,921,837,921,4.394923
construction,912,912,819,912,4.351976
narrow,910,910,867,910,4.342432
